In [31]:
# Pull the data from the Google Drive and Move it to the folder 'New Data'

import os
import shutil

# Source and destination directory paths
source_dir = r"G:\My Drive\Pitching Data\New Data"
destination_dir = r"C:\Users\Biomechanics\PycharmProjects\IRvsPro\files\Pitching Data\New Data"

# Iterate over files in the source directory
for file_name in os.listdir(source_dir):
    if file_name.endswith('.xml'):
        source_file_path = os.path.join(source_dir, file_name)
        destination_file_path = os.path.join(destination_dir, file_name)
        shutil.move(source_file_path, destination_file_path)
        print(f"File '{file_name}' moved to: {destination_file_path}")

print("XML files copied successfully!")


File 'session_MS_12-14-23.xml' moved to: C:\Users\Biomechanics\PycharmProjects\IRvsPro\files\Pitching Data\New Data\session_MS_12-14-23.xml
File 'session_data_MS_12-14-23.xml' moved to: C:\Users\Biomechanics\PycharmProjects\IRvsPro\files\Pitching Data\New Data\session_data_MS_12-14-23.xml
XML files copied successfully!


In [32]:
# This code is *chef's kiss* perfect. Does everything for parsing for variables, iterates over it 
# multiple times, and includes the specific filename for each trial. I love it. 

import sqlite3
import xml.etree.ElementTree as ET
import os
import re

# Connect to the SQLite database
conn = sqlite3.connect("grading_equation_new.db")
cursor = conn.cursor()

# Drop the table if it exists
cursor.execute('DROP TABLE IF EXISTS variables')

# Create the variables table
cursor.execute('''
    CREATE TABLE variables (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        session_data_id TEXT NOT NULL,
        file_name TEXT NOT NULL,
        Pitch TEXT,
        Score REAL,
        Linear_Pelvis_Speed REAL,
        HSS_Footplant REAL,
        Pelvis_Ang_Footplant REAL,
        Trunk_Ang_Footplant REAL,
        Pelvic_Obl REAL,
        Front_Leg_Brace REAL,
        Front_Leg_Var_Val REAL,
        Lead_Leg_GRF_y REAL,
        Lead_Leg_GRF_z REAL,
        Lead_Leg_GRF_x REAL,
        Horizontal_Abduction REAL,
        Shld_ER_Footplant REAL,
        Shld_ER_Max REAL,
        Lateral_Trunk_Tilt REAL,
        Pelvis_Ang_Velo REAL,
        Torso_Ang_Velo REAL,
        Arm_Ang_Velo REAL,
        MPH REAL
    )
''')

# Commit the changes before proceeding to parsing XML files
conn.commit()

# List of XML files to parse (replace this with your actual directory path)
directory_path = "C:\\Users\\Biomechanics\\PycharmProjects\\IRvsPro\\files\\Pitching Data\\New Data"

while True:
    # Get a list of XML files in the specified directory
    xml_files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".xml")]

    # Track the number of new data entries
    new_data_entries = 0

    for file_name in xml_files:
        try:
            # Check if the file has already been processed
            cursor.execute('SELECT COUNT(*) FROM variables WHERE file_name = ?', (file_name,))
            count = cursor.fetchone()[0]

            if count == 0:
                # Parse the XML file
                tree = ET.parse(file_name)
                root = tree.getroot()

                # Extract session_data_id from the file name
                session_data_id_match = re.search(r"session_data_(.+)\.xml", file_name)
                if session_data_id_match:
                    session_data_id = session_data_id_match.group(1)
                else:
                    session_data_id = None

                # Check if session_data_id is None, and skip the file if it is
                if session_data_id is not None:
                    # Iterate over all occurrences of "owner" element with a value containing "Fastball"
                    for owner_element in root.findall('.//owner'):
                        if 'Fastball' in owner_element.get('value', ''):
                            fastball_owner = owner_element
                            # Extract and insert data into the database
                            linear_pelvis_speed = None
                            hss_footplant = None
                            pelvis_ang_fp = None
                            trunk_ang_fp = None
                            pelvis_obl = None
                            front_leg_brace = None
                            front_leg_var_val = None
                            lead_leg_grf_y = None
                            lead_leg_grf_z = None
                            lead_leg_grf_x = None
                            horizontal_abduction = None
                            shld_er_fp = None
                            shld_er_max = None
                            lateral_trunk_tilt = None
                            pelvis_ang_velo = None
                            torso_ang_velo = None
                            arm_ang_velo = None
                            
                            # Extract the value of "owner"
                            pitch_value = fastball_owner.get('value', '')
    
                            for variable_element in fastball_owner.findall('.//name[@value]'):
                                variable_name = variable_element.attrib['value']
                                component_x_element = variable_element.find('./component[@value="X"]')
                                component_y_element = variable_element.find('./component[@value="Y"]')
                                component_z_element = variable_element.find('./component[@value="Z"]')                           
    
                                if variable_name == "MaxPelvisLinearVel_MPH" and component_y_element is not None:
                                    linear_pelvis_speed = float(component_y_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Hip Shoulders Sep@Footstrike" and component_z_element is not None:
                                    hss_footplant = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pelvis_Angle@Footstrike" and component_z_element is not None:
                                    pelvis_ang_fp = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Trunk_Angle@Footstrike" and component_z_element is not None:
                                    trunk_ang_fp = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Lead_Knee_Angle@Footstrike" and component_x_element is not None:
                                    lead_knee_ang_at_footstrike = float(component_x_element.attrib['data'].replace(',', '.'))
                                    lead_knee_ang_at_release = float(root.find('.//name[@value="Lead_Knee_Angle@Release"]/component[@value="X"]').attrib['data'].replace(',', '.'))
                                    front_leg_brace = lead_knee_ang_at_footstrike - lead_knee_ang_at_release
                                if variable_name == "Lead_Knee_Angle@Footstrike" and component_y_element is not None:
                                    lead_knee_var_val_fp = float(component_y_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Lead_Knee_Angle@Release" and component_y_element is not None:
                                    lead_knee_var_val_rel = float(component_y_element.attrib['data'].replace(',', '.'))
                                    # Calculate the difference and store in front_leg_var_val
                                    front_leg_var_val = lead_knee_var_val_fp - lead_knee_var_val_rel
                                if variable_name == "Pelvis_Angle@Footstrike" and component_z_element is not None:
                                    pelvis_obl_fp = float(component_y_element.attrib['data'].replace(',', '.'))
                                    pelvis_angle_release_element = root.find('.//name[@value="Pelvis_Angle@Release"]/component[@value="Y"]')
                                    if pelvis_angle_release_element is not None:
                                        pelvis_obl_release = float(pelvis_angle_release_element.attrib['data'].replace(',', '.'))
                                        pelvis_obl = pelvis_obl_release - pelvis_obl_fp
                                elif variable_name == "Lead_Leg_GRF_min" and component_y_element is not None:
                                    lead_leg_grf_y = abs(float(component_y_element.attrib['data'].replace(',', '.')))
                                elif variable_name == "Lead_Leg_GRF_max" and component_z_element is not None:
                                    lead_leg_grf_z = float(component_z_element.attrib['data'].replace(',', '.'))
                                if variable_name == "Lead_Leg_GRF_max" and component_x_element is not None:
                                    lat_force_max = float(component_x_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Lead_Leg_GRF_min" and component_x_element is not None:
                                    lat_force_min = float(component_x_element.attrib['data'].replace(',', '.'))
                                    # Calculate the sum of absolute values and store in lead_leg_grf_x
                                    lead_leg_grf_x = abs(lat_force_max) + abs(lat_force_min)
                                elif variable_name == "Pitching_Shoulder_Angle@Footstrike" and component_x_element is not None:
                                    horizontal_abduction = abs(float(component_x_element.attrib['data'].replace(',', '.')))
                                if variable_name == "Pitching_Shoulder_Angle@Footstrike":
                                    if component_z_element is not None:
                                        shld_er_fp = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pitching_Shoulder_Angle_Max" and component_z_element is not None:
                                    shld_er_max = abs(float(component_z_element.attrib['data'].replace(',', '.')))
                                elif variable_name == "Trunk_wrt_Pelvis_FE@Release" and component_y_element is not None:
                                    lateral_trunk_tilt = float(component_y_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pelvis_Ang_Vel_max" and component_x_element is not None:
                                    pelvis_ang_velo = float(component_x_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Thorax_Ang_Vel_max" and component_x_element is not None:
                                    torso_ang_velo = float(component_x_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pitching_Humerus_Ang_Vel_max" and component_x_element is not None:
                                    arm_ang_velo = float(component_x_element.attrib['data'].replace(',', '.'))
                                
                                linear_pelvis_speed = linear_pelvis_speed if linear_pelvis_speed is not None else 4
                                hss_footplant = hss_footplant if hss_footplant is not None else 30
                                pelvis_ang_fp = pelvis_ang_fp if pelvis_ang_fp is not None else 60
                                trunk_ang_fp = trunk_ang_fp if trunk_ang_fp is not None else 90
                                pelvis_obl = pelvis_obl if pelvis_obl is not None else 2
                                front_leg_brace = front_leg_brace if front_leg_brace is not None else 8
                                front_leg_var_val = front_leg_var_val if front_leg_var_val is not None else 0
                                lead_leg_grf_y = lead_leg_grf_y if lead_leg_grf_y is not None else .7
                                lead_leg_grf_z = lead_leg_grf_z if lead_leg_grf_z is not None else 1.5
                                lead_leg_grf_x = lead_leg_grf_x if lead_leg_grf_x is not None else .25
                                horizontal_abduction = horizontal_abduction if horizontal_abduction is not None else 20
                                shld_er_fp = shld_er_fp if shld_er_fp is not None else 45
                                lateral_trunk_tilt = lateral_trunk_tilt if lateral_trunk_tilt is not None else 35
                                shld_er_max = shld_er_max if shld_er_max is not None else 160
                                pelvis_ang_velo = pelvis_ang_velo if pelvis_ang_velo is not None else 600
                                torso_ang_velo = torso_ang_velo if torso_ang_velo is not None else 1000
                                arm_ang_velo = arm_ang_velo if arm_ang_velo is not None else 5000
    
                                # Insert the data into the database
                            cursor.execute('''
                                INSERT INTO variables (
                                    session_data_id,
                                    file_name,
                                    Pitch,
                                    Linear_Pelvis_Speed,
                                    HSS_Footplant,
                                    Pelvis_Ang_Footplant,
                                    Trunk_Ang_Footplant,
                                    Pelvic_Obl,
                                    Front_Leg_Brace,
                                    Front_Leg_Var_Val,
                                    Lead_Leg_GRF_y,
                                    Lead_Leg_GRF_z,
                                    Lead_Leg_GRF_x,
                                    Horizontal_Abduction,
                                    Shld_ER_Footplant,
                                    Shld_ER_Max,
                                    Lateral_Trunk_Tilt,
                                    Pelvis_Ang_Velo,
                                    Torso_Ang_Velo,
                                    Arm_Ang_Velo,
                                    MPH
                                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                            ''', (session_data_id, file_name, pitch_value, linear_pelvis_speed, hss_footplant, pelvis_ang_fp, trunk_ang_fp, pelvis_obl, front_leg_brace, front_leg_var_val, lead_leg_grf_y, lead_leg_grf_z, lead_leg_grf_x, horizontal_abduction, shld_er_fp, shld_er_max, lateral_trunk_tilt, pelvis_ang_velo, torso_ang_velo, arm_ang_velo, None))
    
                            # Increment the count of new data entries
                            new_data_entries += 1
                else:
                    print(f"Skipping file {file_name} due to missing session_data_id.")

        except Exception as e:
            # Print the error and the file name where it occurred
            print(f"Error processing file {file_name}: {e}")

    # Commit the changes for each file
    conn.commit()

    # If no new data entries, break out of the loop
    if new_data_entries == 0:
        break

# Close the connection
conn.close()

print("Data inserted into the database.")

Skipping file C:\Users\Biomechanics\PycharmProjects\IRvsPro\files\Pitching Data\New Data\session_MS_12-14-23.xml due to missing session_data_id.
Skipping file C:\Users\Biomechanics\PycharmProjects\IRvsPro\files\Pitching Data\New Data\session_MS_12-14-23.xml due to missing session_data_id.
Data inserted into the database.


In [33]:
# Successfully adds MPH to the appropriate spot in the database

import os
import xml.etree.ElementTree as ET
import re
import sqlite3

# Replace this with your actual directory paths
directory_path = "C:\\Users\\Biomechanics\\PycharmProjects\\IRvsPro\\files\\Pitching Data\\New Data"

# Connect to the database
conn = sqlite3.connect("grading_equation_new.db")
cursor = conn.cursor()

# Get a list of XML files in the specified directory
xml_files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".xml")]

# Track the number of new data entries
new_data_entries = 0

for file_name in xml_files:
    try:
        # Check if the file has already been processed
        cursor.execute('SELECT COUNT(*) FROM variables WHERE file_name = ?', (file_name,))
        count = cursor.fetchone()[0]

        if count == 0:
            # Parse the XML file
            tree = ET.parse(file_name)
            root = tree.getroot()

            # Extract session_data_id from the file name
            session_data_id_match = re.search(r"session_(.+)\.xml", file_name)
            if session_data_id_match:
                session_data_id = session_data_id_match.group(1)
            else:
                session_data_id = None

            # Check if session_data_id is None, and skip the file if it is
            if session_data_id is not None:
                # Iterate over all occurrences of "Measurement" element
                for measurement_element in root.findall('.//Measurement'):
                    pitch_value = measurement_element.get('Filename')
                    used_element = measurement_element.find('./Fields/Used')
                    
                    if pitch_value is not None and used_element is not None:
                        # Replace "qtm" with "c3d" in pitch_value
                        pitch_value = pitch_value.replace("qtm", "c3d")

                        # Print statement for debugging
                        print(f"Filename: {pitch_value}, Used: {used_element.text}")

                        if 'Fastball' in pitch_value and used_element.text == 'True':
                            # Extract the values of "Comments" and "Ball_speed"
                            comments_value_element = measurement_element.find('./Fields/Comments')

                            if comments_value_element is not None:
                                print(f"MPH: {comments_value_element.text}")
                                print(f"Session ID: {session_data_id}")

                                mph_value = float(comments_value_element.text)

                                # Update the MPH value in the "variables" table
                                cursor.execute('''
                                    UPDATE variables
                                    SET MPH = ?
                                    WHERE session_data_id = ? AND Pitch = ?
                                ''', (mph_value, session_data_id, pitch_value))

                                # Increment the new data entries counter
                                new_data_entries += 1
                    else:
                        print("Filename or Used element not found.")

    except Exception as e:
        print(f"Error processing file {file_name}: {e}")

# Commit the changes
conn.commit()

# Close the connection
conn.close()

print(f"Number of new data entries: {new_data_entries}")
print("Data inserted into the database.")

Filename: Fastball RH 1.c3d, Used: False
Filename: Fastball RH 10.c3d, Used: True
MPH: 90.6
Session ID: MS_12-14-23
Filename: Fastball RH 11.c3d, Used: True
MPH: 91.0
Session ID: MS_12-14-23
Filename: Fastball RH 12.c3d, Used: True
MPH: 91.6
Session ID: MS_12-14-23
Filename: Fastball RH 13.c3d, Used: True
MPH: 90.3
Session ID: MS_12-14-23
Filename: Fastball RH 14.c3d, Used: True
MPH: 91.6
Session ID: MS_12-14-23
Filename: Fastball RH 2.c3d, Used: False
Filename: Fastball RH 3.c3d, Used: False
Filename: Fastball RH 4.c3d, Used: False
Filename: Fastball RH 5.c3d, Used: False
Filename: Fastball RH 6.c3d, Used: False
Filename: Fastball RH 7.c3d, Used: False
Filename: Fastball RH 8.c3d, Used: False
Filename: Fastball RH 9.c3d, Used: False
Filename: Functional hip RH 1.c3d, Used: True
Filename: Static 1.c3d, Used: True
Number of new data entries: 5
Data inserted into the database.


In [34]:
# Generates Score

import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("grading_equation_new.db")
cursor = conn.cursor()

# Select all rows from the 'variables' table
cursor.execute('SELECT * FROM variables')
rows = cursor.fetchall()

# Iterate through each row and calculate the score
for row in rows:
    linear_pelvis_speed = row[5]
    hss_footplant = row[6]
    pelvis_ang_fp = row[7]
    trunk_ang_fp = row[8]
    pelvis_obl = row[9]
    front_leg_brace = row[10]
    front_leg_var_val = row[11]
    lead_leg_grf_y = row[12]
    lead_leg_grf_z = row[13]
    lead_leg_grf_x = row[14]
    horizontal_abduction = row[15]
    shld_er_fp = row[16]
    shld_er_max = row[17]
    lateral_trunk_tilt = row[18]
    pelvis_ang_velo = row[19]
    torso_ang_velo = row[20]
    arm_ang_velo = row[21]
    MPH = row[22]

    # Check for None values and replace with 0
    linear_pelvis_speed = linear_pelvis_speed if linear_pelvis_speed is not None else 4
    hss_footplant = hss_footplant if hss_footplant is not None else 30
    pelvis_ang_fp = pelvis_ang_fp if pelvis_ang_fp is not None else 60
    trunk_ang_fp = trunk_ang_fp if trunk_ang_fp is not None else 90
    pelvis_obl = pelvis_obl if pelvis_obl is not None else 2
    front_leg_brace = front_leg_brace if front_leg_brace is not None else 8
    front_leg_var_val = front_leg_var_val if front_leg_var_val is not None else 0
    lead_leg_grf_y = lead_leg_grf_y if lead_leg_grf_y is not None else .7
    lead_leg_grf_z = lead_leg_grf_z if lead_leg_grf_z is not None else 1.5
    lead_leg_grf_x = lead_leg_grf_x if lead_leg_grf_x is not None else .25
    horizontal_abduction = horizontal_abduction if horizontal_abduction is not None else 20
    shld_er_fp = shld_er_fp if shld_er_fp is not None else 45
    lateral_trunk_tilt = lateral_trunk_tilt if lateral_trunk_tilt is not None else 35
    shld_er_max = shld_er_max if shld_er_max is not None else 160
    pelvis_ang_velo = pelvis_ang_velo if pelvis_ang_velo is not None else 600
    torso_ang_velo = torso_ang_velo if torso_ang_velo is not None else 1000
    arm_ang_velo = arm_ang_velo if arm_ang_velo is not None else 5000
    MPH = MPH if MPH is not None else 75

    # Calculate the score using the specified equation
    score = (0.05 * linear_pelvis_speed) + (0.1 * front_leg_brace) + \
            (2 * lead_leg_grf_y) + (.05 * lead_leg_grf_z) + (2 * lead_leg_grf_x) + \
            (0.5 * horizontal_abduction) - (0.05 * lateral_trunk_tilt) + (0.01 * torso_ang_velo) + (.1 * pelvis_obl) + (.2 * trunk_ang_fp) - (.15 * pelvis_ang_fp) + (.05 * shld_er_max) + (.1 * front_leg_var_val) + (.01 * pelvis_ang_velo) + MPH

    # Update the 'Score' column in the database
    cursor.execute('UPDATE variables SET Score = ? WHERE id = ?', (score, row[0]))

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Scores updated in the database.")

Scores updated in the database.


In [35]:
# Generates percentile graphs and exports both and PDF and Word Doc for the person

import os
import sqlite3
import scipy.stats as stats
import matplotlib.pyplot as plt
from docx import Document
from matplotlib.backends.backend_pdf import PdfPages

# Connect to the SQLite databases
conn_new = sqlite3.connect("grading_equation_new.db")
conn_reference = sqlite3.connect("grading_equation_reference_data_Pro.db")
cursor_new = conn_new.cursor()
cursor_reference = conn_reference.cursor()

# Define the directory to save exported graphs
export_dir = "C:\\Users\\Biomechanics\\PycharmProjects\\IRvsPro\\data\\Exported Data"

try:
    # Get the session_data_id for naming the Word document
    cursor_new.execute('SELECT session_data_id FROM variables LIMIT 1')
    session_data_id = cursor_new.fetchone()[0]

    # Define the filename for the Word document
    docx_filename = os.path.join(export_dir, f"plots_{session_data_id}.docx")

    # Create a Word document to save the plots
    doc = Document()

    # Create a PDF file to save the plots
    pdf_filename = os.path.join(export_dir, f"plots_{session_data_id}.pdf")
    pdf_pages = PdfPages(pdf_filename)

    # Get the list of columns in the new database
    cursor_new.execute("PRAGMA table_info(variables)")
    column_names = [row[1] for row in cursor_new.fetchall() if row[1] not in ('id', 'session_data_id', 'file_name', 'Pitch')]
    
    # Retrieve the row with the highest score from the new database
    cursor_new.execute('SELECT * FROM variables ORDER BY Score DESC LIMIT 1')
    highest_score_row = cursor_new.fetchone()
    
    # Get the values from the highest scoring row
    highest_score_values = highest_score_row[4:]  # Exclude 'id', 'session_data_id', 'file_name', and 'Pitch'
    
    for i, column_name in enumerate(column_names):
        # Retrieve the reference values from the reference database
        cursor_reference.execute(f"SELECT {column_name} FROM variables")
        reference_values = cursor_reference.fetchall()
        reference_values = [value[0] for value in reference_values if value[0] is not None]
        
        # Calculate the percentile for the current variable
        percentile = stats.percentileofscore(reference_values, highest_score_values[i], kind='mean')
        
        # Create a new figure with black background
        fig = plt.figure(facecolor='#181818')
        
        # Create axes with charcoal background
        ax = fig.add_subplot(111, facecolor='#303030')
        
        # Adjust the size of the figure
        plt.subplots_adjust(bottom=0.2)  # Increase the bottom margin to create space for annotation

        # Create a histogram for the reference values with blue bars
        ax.hist(reference_values, bins=20, color='cornflowerblue', alpha=0.7, label='Reference Data')
        
        # Add a vertical line for the highest scoring value
        ax.axvline(x=highest_score_values[i], color='red', linestyle='dashed', linewidth=2, label='Highest Score')
        
        # Add labels and title with white color
        ax.set_xlabel(column_name, color='slategrey')
        ax.set_ylabel('Frequency', color='slategrey')
        ax.set_title(f'Histogram of {column_name}', color='lightgrey')
        
        # Set the color of the gridlines
        ax.grid(color='dimgrey')

        # Set the color of ticks and spines
        ax.tick_params(axis='x', colors='dimgrey')
        ax.tick_params(axis='y', colors='dimgrey')
        ax.spines['bottom'].set_color('slategrey')
        ax.spines['top'].set_color('slategrey')
        ax.spines['left'].set_color('slategrey')
        ax.spines['right'].set_color('slategrey')
        
        # Add legend with white color
        ax.legend(facecolor='black', edgecolor='grey', prop={'size': 'small'}, labelcolor='grey')
        
        # Round the percentile score and the corresponding value
        rounded_percentile = round(percentile, 2)
        rounded_value = round(highest_score_values[i], 2)

        # Add annotation for percentile score and corresponding value
        ax.annotate(f'Percentile: {rounded_percentile}%\nValue: {rounded_value}',
            xy=(highest_score_values[i], 0),
            xytext=(1.0, -0.15),  # Adjusted position below x-axis label in bottom right
            textcoords='axes fraction',  # Coordinates relative to axes
            ha='right', va='top',  # Horizontal and vertical alignment
            fontsize=12, color='white')
        
        # Save the graph to a file (PNG)
        filename = os.path.join(export_dir, f'{column_name}.png')
        plt.savefig(filename)
        
        # Add the graph to the Word document
        doc.add_picture(filename)
        
        # Save the graph to the PDF
        pdf_pages.savefig(fig)
        
        # Clear the current figure to create a new one for the next variable
        plt.clf()
        
        print(f"Graph for {column_name} exported successfully.")
        
    # Close the PDF pages
    pdf_pages.close()

    # Save the Word document
    doc.save(docx_filename)
    
except sqlite3.Error as e:
    print("Error:", e)
    
finally:
    # Close the connections
    conn_new.close()
    conn_reference.close()


Graph for Score exported successfully.
Graph for Linear_Pelvis_Speed exported successfully.
Graph for HSS_Footplant exported successfully.
Graph for Pelvis_Ang_Footplant exported successfully.
Graph for Trunk_Ang_Footplant exported successfully.
Graph for Pelvic_Obl exported successfully.
Graph for Front_Leg_Brace exported successfully.
Graph for Front_Leg_Var_Val exported successfully.
Graph for Lead_Leg_GRF_y exported successfully.
Graph for Lead_Leg_GRF_z exported successfully.
Graph for Lead_Leg_GRF_x exported successfully.
Graph for Horizontal_Abduction exported successfully.
Graph for Shld_ER_Footplant exported successfully.
Graph for Shld_ER_Max exported successfully.
Graph for Lateral_Trunk_Tilt exported successfully.
Graph for Pelvis_Ang_Velo exported successfully.
Graph for Torso_Ang_Velo exported successfully.
Graph for Arm_Ang_Velo exported successfully.
Graph for MPH exported successfully.


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [ ]:
import os
import shutil

# Source directory path
source_dir = r"C:\Users\Biomechanics\PycharmProjects\IRvsPro\files\Pitching Data\New Data"

# Destination directory paths
kinematic_data_dir = r"C:\Users\Biomechanics\PycharmProjects\IRvsPro\files\Pitching Data\Kinematic Data\Pro"
mph_data_dir = r"C:\Users\Biomechanics\PycharmProjects\IRvsPro\files\Pitching Data\MPH Data\Pro"

# Iterate over files in the source directory
for file_name in os.listdir(source_dir):
    if 'data' in file_name.lower():
        source_file_path = os.path.join(source_dir, file_name)
        destination_file_path = os.path.join(kinematic_data_dir, file_name)
        shutil.move(source_file_path, destination_file_path)
        print(f"File '{file_name}' moved to: {destination_file_path}")
    else:
        source_file_path = os.path.join(source_dir, file_name)
        destination_file_path = os.path.join(mph_data_dir, file_name)
        shutil.move(source_file_path, destination_file_path)
        print(f"File '{file_name}' moved to: {destination_file_path}")

print("Files moved successfully!")


In [20]:
# Generates percentiles

import os
import sqlite3
import scipy.stats as stats

# Connect to the SQLite databases
conn_new = sqlite3.connect("grading_equation_new.db")
conn_reference = sqlite3.connect("grading_equation_reference_data_Pro.db")
cursor_new = conn_new.cursor()
cursor_reference = conn_reference.cursor()

try:
    # Get the list of columns in the new database
    cursor_new.execute("PRAGMA table_info(variables)")
    column_names = [row[1] for row in cursor_new.fetchall() if row[1] not in ('id', 'session_data_id', 'file_name', 'Pitch')]
    
    # Retrieve the row with the highest score from the new database
    cursor_new.execute('SELECT * FROM variables ORDER BY Score DESC LIMIT 1')
    highest_score_row = cursor_new.fetchone()
    
    # Get the values from the highest scoring row
    highest_score_values = highest_score_row[4:]  # Exclude 'id', 'session_data_id', 'file_name', and 'Pitch'
    
    # Iterate over each column starting from index 0
    for i, column_name in enumerate(column_names):
        # Retrieve the reference values from the reference database
        cursor_reference.execute(f"SELECT {column_name} FROM variables")
        reference_values = cursor_reference.fetchall()
        reference_values = [value[0] for value in reference_values if value[0] is not None]
        
        # Calculate the percentile of the variable value compared to reference data
        percentile = stats.percentileofscore(reference_values, highest_score_values[i], kind='mean')
        
        # Print the percentile
        print(f"Percentile for {column_name}: {percentile}%")
        
except sqlite3.Error as e:
    print("Error:", e)
    
finally:
    # Close the connections
    conn_new.close()
    conn_reference.close()

Percentile for Score: 7.817589576547231%
Percentile for Linear_Pelvis_Speed: 0.0%
Percentile for HSS_Footplant: 3.2573289902280127%
Percentile for Pelvis_Ang_Footplant: 27.035830618892508%
Percentile for Trunk_Ang_Footplant: 3.2573289902280127%
Percentile for Pelvic_Obl: 98.37133550488599%
Percentile for Front_Leg_Brace: 66.12377850162866%
Percentile for Front_Leg_Var_Val: 51.79153094462541%
Percentile for Lead_Leg_GRF_y: 29.96742671009772%
Percentile for Lead_Leg_GRF_z: 0.3257328990228013%
Percentile for Lead_Leg_GRF_x: 81.75895765472312%
Percentile for Horizontal_Abduction: 22.80130293159609%
Percentile for Shld_ER_Footplant: 95.43973941368077%
Percentile for Shld_ER_Max: 0.0%
Percentile for Lateral_Trunk_Tilt: 2.6058631921824102%
Percentile for Pelvis_Ang_Velo: 35.50488599348534%
Percentile for Torso_Ang_Velo: 100.0%
Percentile for Arm_Ang_Velo: 5.537459283387622%
Percentile for MPH: 0.0%
